Basics of Python

In [11]:
def add_something(a, b):
    print(f"Adding {a} and {b} together...")
    return a + b

add_something(5, 10), add_something(**{"a": 5, "b": 10}), add_something(*[5, 10])# a=5, b=10, add_something(*[5, 10])

Adding 5 and 10 together...
Adding 5 and 10 together...
Adding 5 and 10 together...


(15, 15, 15)

Did you know? In Python, `**` in a function call means **“take this mapping (usually a dict) and expand it into keyword arguments.”**

### What’s happening in your call

```python
add_something(**{"a": 7, "b": 12})
```

Python sees a dict with keys `"a"` and `"b"`, and expands it as if you had written:

```python
add_something(a=7, b=12)
```

So it’s not passing the dict as a single argument. It’s **unpacking** it into named parameters.

### Why it works

Because your function is defined with parameter names `a` and `b`:

```python
def add_something(a, b):
```

…and the dict has matching keys:

```python
{"a": 7, "b": 12}
```

So `a` gets `7`, `b` gets `12`.

### Rules (the important bits)

1. **Keys must be strings** (valid keyword names, effectively):

   ```python
   add_something(**{1: 7, 2: 12})  # TypeError
   ```

2. **Keys must match parameter names**, unless the function accepts arbitrary keywords:

   ```python
   add_something(**{"x": 7, "b": 12})
   # TypeError: got an unexpected keyword argument 'x'
   ```

3. You can mix normal args + `**` (but don’t give the same value twice):

   ```python
   add_something(7, **{"b": 12})      # ok -> a=7, b=12
   add_something(a=7, **{"a": 9})     # TypeError: multiple values for 'a'
   ```

4. You can also merge multiple `**` expansions:

   ```python
   add_something(**{"a": 7}, **{"b": 12})
   ```

### Related: `*` vs `**`

* `*` unpacks a sequence into **positional arguments**:

  ```python
  add_something(*[7, 12])   # same as add_something(7, 12)
  ```

* `**` unpacks a mapping into **keyword arguments**:

  ```python
  add_something(**{"a": 7, "b": 12})  # same as add_something(a=7, b=12)
  ```

If you want, I can show how this interacts with `def f(*args, **kwargs)` which is where `**` really becomes obvious.


# Tool Calling

In [7]:
import os, json
from dotenv import load_dotenv

import textwrap



def pretty_print(*args):
    text = " ".join(str(arg) for arg in args)
    try:
        print(textwrap.fill(text, width=80))
    except Exception as e:
        print(text)  # fallback to normal print if text is not a string

        

load_dotenv('/Users/shivam13juna/Documents/scaler/iitr_classes/llm_ref/openai_key.env')  # reads .env file in the current directory

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError(
        "OPENAI_API_KEY not found! "
        "Make sure you have a .env file with: OPENAI_API_KEY=sk-..."
    )

pretty_print("API key loaded successfully.")

API key loaded successfully.


In [8]:
import truststore
truststore.inject_into_ssl()

#This is optional. I use VPN in my computer. Why I have to use this

from openai import OpenAI

client = OpenAI(api_key=api_key)
pretty_print("OpenAI client ready.")

MODEL  = "gpt-5-nano"  

OpenAI client ready.


In [9]:
# The model PREDICTS math — it doesn't COMPUTE it

response = client.responses.create(
    model=MODEL,
    input="What is 1247 * 83 + 19 / 3.7? Give me answer in one line (max 10 words), the exact number, upto 2 decimal places.",
    reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
print("Model says:", response.output_text)

# What Python actually computes
import math
actual = 1247 * 83 + 19 / 3.7
print(f"Actual answer: {actual}")


Model says: 104,801.70
Actual answer: 103506.13513513513


In [12]:
# The model has NO access to live data — what does it do?

response = client.responses.create(
    model=MODEL,
    input="What is the weather in bengaluru right now? Give me the exact temperature.",
    reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
print(response.output_text)

I don’t have real-time weather data. Please check a live source like a weather app or website (e.g., Weather.com, AccuWeather, or Google Weather) for the exact current temperature in Bengaluru. If you share a link or data, I can help interpret it.


In [13]:
# Can the model actually DO something in the real world?

response = client.responses.create(
    model=MODEL,
    input="Send an email to john@example.com saying 'Meeting moved to 3pm'.",
    reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
print(response.output_text)

I can’t send emails directly, but here’s a ready-to-send message you can use:

To: john@example.com
Subject: Meeting time updated
Body:
Meeting moved to 3pm

If you want, I can format it for your email client or help you draft a more detailed note.


# Tool Calling

1. Create a tool
2. How to register a tool in LLM
3. How to call a tool in LLM
4. How to parse the output of a tool in LLM

https://developers.openai.com/api/docs/guides/function-calling/

---
## 1.3 · Anatomy of a Tool Definition

Let's break down exactly what we just wrote. Every tool definition has these parts:

```python
{
    "type": "function",          # Always "function" for custom tools
    "name": "add",               # What the model calls it by
    "description": "Add two numbers together.",   # Helps the model decide WHEN to use it
    "parameters": {              # JSON Schema describing the arguments
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],           # Which fields are mandatory
        "additionalProperties": False,    # No extra fields allowed (needed for strict)
    },
    "strict": True,              # Guarantee arguments match schema exactly
}
```

**Think of it like a contract:**  
- The `description` tells the model *"here's what this tool does — use it when relevant"*
- The `parameters` schema tells the model *"here's exactly what arguments to generate"*
- `strict: True` tells the API *"enforce this schema — reject anything that doesn't match"*

> The model never sees your Python function code. It only sees this definition.


In [ ]:
#3, 4, 5

#add(add(3, 4), 5)

#### Ex 1

In [14]:
# Define a simple "add" tool — we'll explain the structure in detail next
add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the add tool for any math. Never compute math yourself.",
    input="What is 7 + 12?",
    tools=[add_tool],
)


for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

  Type: reasoning
  Type: function_call
  Function name: add
  Arguments: {"a":7,"b":12}
  Call ID: call_RGINpien0zDC0LMr1hC84g1M
 Response:


#### Ex 2

In [15]:
# Define a simple "add" tool — we'll explain the structure in detail next
add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the add tool for any math. Never compute math yourself.",
    input="What is 7 + 12 + 5?",
    tools=[add_tool],
)

# add(add(7, 12), 5)
for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

  Type: reasoning
  Type: function_call
  Function name: add
  Arguments: {"a":7,"b":12}
  Call ID: call_wFbyyQa91nKUrgolbwRbdtT2
 Response:


#### Ex 3

In [16]:
# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the add tool for any math. Never compute math yourself.",
    input="Why is Earth round?",
    tools=[add_tool],
)


# Let's inspect what came back
print("Output items from the model:")
print("-" * 40)
for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

Output items from the model:
----------------------------------------
  Type: reasoning
  Type: message
 Response: Earth isn’t a perfect ball, but it is round for the same reason many
planets are: gravity.  - When a body has enough mass, gravity pulls every bit of
material toward the center. Over time this tends to form a sphere, because a
sphere is the shape that minimizes gravitational potential energy. - Earth
formed from molten rock and metal, so its material could flow. Gravity reshaped
it into a hydrostatic equilibrium form, i.e., a rounded shape. - Because Earth
spins, rotation flings material outward at the equator a bit, making it slightly
wider there and a bit flattened at the poles. The result is an oblate spheroid
(a “flattened sphere”).   - Equatorial radius: about 6378 km   - Polar radius:
about 6357 km   - Flattening roughly 1/298  Local features like mountains and
ocean trenches create bumps, but the overall shape is that of a rounded,
slightly squashed sphere. The prec

#### Ex 5

In [17]:
# Define a get_weather tool
weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the current temperature for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City name, e.g. 'Tel Aviv', 'London'"
            },
        },
        "required": ["location"],
        "additionalProperties": False,
    },
    "strict": True,
}

# Same question as before — but now the model has a tool!
response = client.responses.create(
    model=MODEL,
    input="What's the weather in Paris right now?",
    tools=[weather_tool],
)

# Let's inspect what came back
print("Output items from the model:")
print("-" * 40)
for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

Output items from the model:
----------------------------------------
  Type: reasoning
  Type: function_call
  Function name: get_weather
  Arguments: {"location":"Paris"}
  Call ID: call_lhVJk6jYCuVPIPHNyVqAR4mm
 Response:


# The Complete Flow

## Example 1

In [ ]:
# Define a simple "add" tool — we'll explain the structure in detail next
add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the add tool for any math. Never compute math yourself.",
    input="What is 7 + 12 ?",
    tools=[add_tool],
)


for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

  Type: reasoning
  Type: function_call
  Function name: add
  Arguments: {"a":7,"b":12}
  Call ID: call_kTXSKoGfjDBfOPmGeI8ioR0T
 Response:


In [28]:
def add(a, b):
    return a + b

function_calls = [item for item in response.output if item.type == "function_call"]

for function in function_calls:
    if function.name == "add":
        args = json.loads(function.arguments)
        #   print("args:", args, type(args), function.arguments, type(function.arguments))
        result = add(**args)

        print(f"Computed {args['a']} + {args['b']} = {result} for call ID {function.call_id}")

Computed 7 + 12 = 19 for call ID call_kTXSKoGfjDBfOPmGeI8ioR0T


In [ ]:
final = client.responses.create(
    model=MODEL,
    instructions="Always use the add tool for math. Never compute yourself.",
    tools=[add_tool],
	previous_response_id=response.id,   # links to the previous response
    input=[
        {"type": "function_call_output", "call_id": function.call_id, "output": str(result)}
    ],
)
## EE ##a;slkjdfasl;djkf l;
for item in final.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + final.output_text)

  Type: message
 Response: 19


## Example 2

In [34]:
# Define a simple "add" tool — we'll explain the structure in detail next
add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the add tool for any math. Never compute math yourself.",
    input="What is 7 + 12 + 5 ?",
    tools=[add_tool],
)


for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

  Type: reasoning
  Type: function_call
  Function name: add
  Arguments: {"a":7,"b":12}
  Call ID: call_y2znmuDjYJGXjbLRxGct4n0I
 Response:


In [35]:
def add(a, b):
    return a + b

function_calls = [item for item in response.output if item.type == "function_call"]

for function in function_calls:
    if function.name == "add":
        args = json.loads(function.arguments)
        #   print("args:", args, type(args), function.arguments, type(function.arguments))
        result = add(**args)

        print(f"Computed {args['a']} + {args['b']} = {result} for call ID {function.call_id}")

Computed 7 + 12 = 19 for call ID call_y2znmuDjYJGXjbLRxGct4n0I


In [36]:
final = client.responses.create(
    model=MODEL,
    instructions="Always use the add tool for math. Never compute yourself.",
    tools=[add_tool],
	previous_response_id=response.id,   # links to the previous response
    input=[
        {"type": "function_call_output", "call_id": function.call_id, "output": str(result)}
    ],
)
## EE ##a;slkjdfasl;djkf l;
for item in final.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + final.output_text)

  Type: function_call
  Function name: add
  Arguments: {"a":19,"b":5}
  Call ID: call_snvRxjhzNhkQ61X9vXtSO2IC
 Response:


In [38]:
def add(a, b):
    return a + b

function_calls = [item for item in final.output if item.type == "function_call"]

for function in function_calls:
    if function.name == "add":
        args = json.loads(function.arguments)
        #   print("args:", args, type(args), function.arguments, type(function.arguments))
        result = add(**args)

        print(f"Computed {args['a']} + {args['b']} = {result} for call ID {function.call_id}")

Computed 19 + 5 = 24 for call ID call_snvRxjhzNhkQ61X9vXtSO2IC


In [39]:
final_v2 = client.responses.create(
    model=MODEL,
    instructions="Always use the add tool for math. Never compute yourself.",
    tools=[add_tool],
	previous_response_id=final.id,   # links to the previous response
    input=[
        {"type": "function_call_output", "call_id": function.call_id, "output": str(result)}
    ],
)
## EE ##a;slkjdfasl;djkf l;
for item in final_v2.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + final_v2.output_text)

  Type: message
 Response: 24


## Example 3

In [40]:

# Adding more tools doesn't change the model's behavior if the question doesn't require them

add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

sub_tool = {
    "type": "function",
    "name": "subtract",
    "description": "Subtract two numbers and return the difference.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

mul_tool = {
    "type": "function",
    "name": "multiply",
    "description": "Multiply two numbers together and return the product.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


div_tool = {
    "type": "function",
    "name": "divide",
    "description": "Divide two numbers and return the answer.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "Numerator (dividend)"},
            "b": {"type": "number", "description": "Denominator (divisor)"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b

def divide(a, b):
    if b == 0:
        return "Error: Division by zero"
    return a / b


DISPATCH = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
    "divide": divide,
}

TOOLS = [add_tool, sub_tool, mul_tool, div_tool]


In [42]:
DEV_POLICY = "Use the tools for any math. Never compute math yourself."


def answer_with_math_tools_verbose(user_question: str) -> str:
    print("══════════════════════════════════════════════════════")
    print("START: User question")
    print("  ", user_question)
    print("══════════════════════════════════════════════════════")

    # Round 1: seed conversation with developer policy + user question
    resp = client.responses.create(
        model=MODEL,
        input=[
            {"role": "developer", "content": DEV_POLICY},
            {"role": "user", "content": user_question},
        ],
        tools=TOOLS
    )

    round_num = 1

    while True:
        print(f"\n══════════════════════════════════════════════════════")
        print(f"ROUND {round_num}: Model response items")
        print("══════════════════════════════════════════════════════")

        # Show every output item type
        for i, item in enumerate(resp.output):
            print(f"[{i}] type = {item.type}")

            if item.type == "function_call":
                print(f"    name     = {item.name}")
                print(f"    call_id  = {item.call_id}")
                print(f"    arguments(raw JSON string) = {item.arguments}")

            elif item.type == "message":
                # Some SDKs represent assistant text as message items
                # output_text is the easiest way to get the final combined text.
                try:
                    print(f"    content = {item.content}")
                except Exception:
                    pass

        calls = [item for item in resp.output if item.type == "function_call"]

        # If no tool calls, we’re done; return final user-facing text
        if not calls:
            print("\n✅ No function calls. Final assistant text:")
            print(resp.output_text)
            return resp.output_text

        print("\n→ Model requested tool calls:")
        for call in calls:
            print(f"  - {call.name}({call.arguments})  [call_id={call.call_id}]")

        # Execute all calls and prepare tool outputs
        tool_outputs = []
        print("\n→ Executing tools locally (your server/app):")
        for call in calls:
            fn = DISPATCH.get(call.name)
            args = json.loads(call.arguments)

            try:
                result = fn(**args)
                payload = {"ok": True, "result": result}
                print(f"  ✓ {call.name}(**{args}) -> {result}")
            except Exception as e:
                payload = {"ok": False, "error": str(e)}
                print(f"  ✗ {call.name}(**{args}) -> ERROR: {e}")

            tool_outputs.append({
                "type": "function_call_output",
                "call_id": call.call_id,
                # output is a string; keep it JSON for readability + structure
                "output": json.dumps(payload),
            })

        print("\n→ Sending tool outputs back to the model:")
        for out in tool_outputs:
            pretty_print(out)

        # Continue conversation, ONLY sending tool outputs (chained with previous_response_id)
        resp = client.responses.create(
            model=MODEL,
            previous_response_id=resp.id,
            input=tool_outputs,
            tools=TOOLS,
            reasoning={"effort": "minimal"},
        )

        round_num += 1

# Example
final_text = answer_with_math_tools_verbose("What is 1247 * 83 + 19 / 3.7?")

══════════════════════════════════════════════════════
START: User question
   What is 1247 * 83 + 19 / 3.7?
══════════════════════════════════════════════════════

══════════════════════════════════════════════════════
ROUND 1: Model response items
══════════════════════════════════════════════════════
[0] type = reasoning
[1] type = function_call
    name     = multiply
    call_id  = call_FXqSp8dEA984geRAPIAO0BoE
    arguments(raw JSON string) = {"a":1247,"b":83}
[2] type = function_call
    name     = divide
    call_id  = call_ej572fm9Y0ZMViNWVB5bACVd
    arguments(raw JSON string) = {"a":19,"b":3.7}

→ Model requested tool calls:
  - multiply({"a":1247,"b":83})  [call_id=call_FXqSp8dEA984geRAPIAO0BoE]
  - divide({"a":19,"b":3.7})  [call_id=call_ej572fm9Y0ZMViNWVB5bACVd]

→ Executing tools locally (your server/app):
  ✓ multiply(**{'a': 1247, 'b': 83}) -> 103501
  ✓ divide(**{'a': 19, 'b': 3.7}) -> 5.135135135135135

→ Sending tool outputs back to the model:
{'type': 'function_cal